# Pointer-over-Heads Transformer (PoT)

**Dynamic multi-head attention with adaptive routing for dependency parsing**

Repository: https://github.com/Eran-BA/PoT

This notebook demonstrates training, evaluation, and visualization of the PoH architecture.
